In [1]:
import radarsimpy

print("`RadarSimPy` used in this example is version: " + str(radarsimpy.__version__))

`RadarSimPy` used in this example is version: 14.1.0


In [2]:
# Import necessary modules for radar simulation
import numpy as np
from radarsimpy import Radar, Transmitter, Receiver

In [3]:
# Define antenna gain
antenna_gain = 20  # dBi - High-gain directional antenna

### Azimuth Pattern (Horizontal Plane) ###

# Define azimuth angle array: -20° to +20° in 1° steps
az_angle = np.arange(-20, 21, 1)  # 41 points

# Azimuth pattern: cos^500 taper with 20 dBi peak gain
# Very narrow beamwidth (~2° @ -3dB) with low sidelobes
az_pattern = 20 * np.log10(np.cos(az_angle / 180 * np.pi) ** 500) + antenna_gain  # dB

### Elevation Pattern (Vertical Plane) ###

# Define elevation angle array: -20° to +20° in 1° steps
el_angle = np.arange(-20, 21, 1)  # 41 points

# Elevation pattern: cos^400 taper with 20 dBi peak gain
# Narrow beamwidth (~2.5° @ -3dB)
el_pattern = 20 * np.log10((np.cos(el_angle / 180 * np.pi)) ** 400) + antenna_gain  # dB

In [4]:
# Import constants
from scipy import constants

# Speed of light constant
light_speed = constants.c  # 299,792,458 m/s

### Calculate Pulsed Radar Waveform Parameters ###

# Performance Requirements
max_range = 5000  # Maximum unambiguous range: 5000 m (5 km)
range_res = 50  # Required range resolution: 50 m

# Derived Waveform Parameters
pulse_bw = light_speed / (2 * range_res)  # Pulse bandwidth: c/(2*ΔR) = 3 MHz
pulse_width = 1 / pulse_bw  # Pulse width: 1/B ≈ 333 ns
prf = light_speed / (2 * max_range)  # PRF: c/(2*R_max) = 30 kHz
prp = 1 / prf  # PRP: 1/PRF ≈ 33.3 μs

# Radar Parameters
fs = 6e6  # Sampling rate: 6 MHz (> 2 × pulse_bw for Nyquist)
fc = 10e9  # Carrier frequency: 10 GHz (X-band)
num_pulse = 10  # Number of pulses: 10 (for non-coherent integration)

# Calculate sample counts
total_samples = int(prp * fs * num_pulse)  # Total samples in observation
sample_per_pulse = int(prp * fs)  # Samples per pulse

### Create Rectangular Pulse Envelope ###

# Create time array for single pulse period
mod_t_single_pulse = np.arange(0, sample_per_pulse) / fs  # Time vector (seconds)

# Initialize amplitude modulation array (all zeros initially)
amp_single_pulse = np.zeros_like(mod_t_single_pulse)

# Set pulse-on period: amplitude = 1 for duration of pulse_width
amp_single_pulse[mod_t_single_pulse <= pulse_width] = (
    1  # Rectangular pulse (first ~333 ns)
)

# Repeat the pulse pattern for all pulses
amp = np.tile(amp_single_pulse, num_pulse)  # Repeat amplitude pattern num_pulse times
mod_t = np.arange(0, len(amp)) / fs  # Time array for entire pulse train


### Define Transmitter Channel ###

# Configure TX channel with antenna pattern and pulse modulation
tx_channel = dict(
    location=(0, 0, 0),  # Antenna position at origin
    azimuth_angle=az_angle,  # Azimuth angles array
    azimuth_pattern=az_pattern,  # Azimuth gain pattern (dB)
    elevation_angle=el_angle,  # Elevation angles array
    elevation_pattern=el_pattern,  # Elevation gain pattern (dB)
    amp=amp,  # Pulse amplitude modulation
    mod_t=mod_t,  # Modulation time array
)

### Create Pulsed Radar Transmitter ###

tx = Transmitter(
    f=fc,  # Carrier frequency: 10 GHz
    t=1 / prf * num_pulse,  # Total observation time: PRP × num_pulse
    tx_power=67,  # Transmit power: 67 dBm ≈ 5 kW peak power
    pulses=1,  # Number of pulse sequences: 1
    channels=[tx_channel],  # Transmitter antenna configuration
)

### Visualize Transmit Pulse

Display the rectangular pulse envelope showing the short pulse duration.

In [5]:
import plotly.graph_objs as go

# Create figure for pulse visualization
fig = go.Figure()

# Plot pulse envelope showing all 10 pulses
fig.add_trace(
    go.Scatter(
        x=mod_t * 1e6,  # Convert to microseconds
        y=amp * 1,  # Pulse amplitude
        name="Transmit Pulse",
        line=dict(color="green", width=2),
        fill="tozeroy",
        fillcolor="rgba(0,255,0,0.3)",
    )
)

# Configure plot layout
fig.update_layout(
    title="Rectangular Transmit Pulse Train: 10 Pulses, 333 ns Each",
    yaxis=dict(title="Amplitude (normalized)", gridcolor="lightgray"),
    xaxis=dict(title="Time (μs)", gridcolor="lightgray"),
    height=500,
)

# uncomment this to display interactive plot
fig.show()

In [6]:
# Define receiver channel with same antenna pattern (monostatic)
rx_channel = dict(
    location=(0, 0, 0),  # Receiver position at origin (same as TX)
    azimuth_angle=az_angle,  # Same azimuth pattern as TX
    azimuth_pattern=az_pattern,  # Same azimuth gain
    elevation_angle=el_angle,  # Same elevation pattern
    elevation_pattern=el_pattern,  # Same elevation gain
)

### Create Radar Receiver

Define receiver with sampling and gain parameters.

In [7]:
# Create pulsed radar receiver
rx = Receiver(
    fs=fs,  # Sampling rate: 6 MHz
    noise_figure=12,  # Noise figure: 12 dB
    rf_gain=20,  # RF gain: 20 dB (LNA)
    load_resistor=500,  # Load resistance: 500 Ω
    baseband_gain=80,  # Baseband gain: 80 dB
    channels=[rx_channel],  # Receiver antenna configuration
)

### Create Radar System

Combine transmitter and receiver to form the complete pulsed radar.

In [8]:
# Create complete pulsed radar system
radar = Radar(transmitter=tx, receiver=rx, device="cpu")

In [9]:
# Configure Target 1: 6 km range, stationary
target_1 = dict(
    location=(6000, 0, 0),  # Position: 6000m range
    speed=(0, 0, 0),  # Velocity: stationary
    rcs=10,  # Radar cross section: 10 dBsm (small aircraft)
    phase=0,  # Initial phase: 0 degrees
)

# Configure Target 2: 3 km range, stationary
target_2 = dict(
    location=(3000, 0, 0),  # Position: 3000m range (3000m spacing from target 1)
    speed=(0, 0, 0),  # Velocity: stationary
    rcs=10,  # Radar cross section: 10 dBsm (same as target 1)
    phase=0,  # Initial phase: 0 degrees
)

# Combine targets for simulation
targets = [target_1, target_2]

In [10]:
# Import radar simulator
from radarsimpy.simulator import sim_radar

# Simulate pulsed radar with two stationary targets
data = sim_radar(radar, targets)

# Extract simulation results
timestamp = data["timestamp"]  # Time array [1, 10, samples]
baseband = data["baseband"]  # Complex I/Q + noise [1, 10, samples]

print(f"Simulation complete:")
print(f"  Baseband shape: {baseband.shape}")

Simulation complete:
  Baseband shape: (1, 1, 2001)


### Visualize Baseband Signals

Display time-domain I/Q signals showing pulse returns and noise.

In [11]:
# Create figure for baseband I/Q visualization
fig = go.Figure()

# Plot In-phase (I) component
fig.add_trace(
    go.Scatter(
        x=timestamp[0, 0, :] * 1e6,  # Convert to microseconds
        y=np.real(baseband[0, 0, :]),  # Real part (I channel)
        name="I (In-phase)",
        line=dict(color="blue", width=1),
    )
)

# Plot Quadrature (Q) component
fig.add_trace(
    go.Scatter(
        x=timestamp[0, 0, :] * 1e6,  # Convert to microseconds
        y=np.imag(baseband[0, 0, :]),  # Imaginary part (Q channel)
        name="Q (Quadrature)",
        line=dict(color="red", width=1),
    )
)

# Configure plot layout
fig.update_layout(
    title="Baseband I/Q Signals: Pulse Returns from Two Targets",
    yaxis=dict(title="Amplitude (V)", gridcolor="lightgray"),
    xaxis=dict(title="Time (μs)", gridcolor="lightgray"),
    height=500,
    legend=dict(x=0.02, y=0.98),
)


fig.show()